
# 2D CNN Action Classification with Fusion (Early & Late) on UCF50 — PyTorch

**Mục tiêu:** Xây dựng pipeline phân loại hành động (action classification) từ video bằng **2D CNN** với các cơ chế **Fusion** dạng *early* và *late* có thể chọn bằng cấu hình.  
**Bộ dữ liệu:** [UCF50](https://www.crcv.ucf.edu/data/UCF50.php) — tổ chức theo thư mục `root/ClassName/*.avi`.

**Bạn sẽ nhận được:**
- Bộ **DataLoader** đọc video (`OpenCV`), lấy mẫu khung hình, tính **Optical Flow** (Farneback/TV-L1) có hỗ trợ **cache** ra đĩa.
- Các mô hình:
  - `Early-Channel Fusion`: ghép RGB + Flow theo **chiều kênh** (C) và **sửa conv1** để nhận nhiều kênh.
  - `Early-Feature Fusion`: hai backbone riêng cho RGB/Flow, **nối đặc trưng** trước classifier.
  - `Late Fusion`: hai backbone riêng, **trung bình/weighted** các **logits** (hoặc xác suất).
- Vòng lặp huấn luyện có **Early Stopping**, **LR Scheduler**, lưu **best model**, đánh giá và **Confusion Matrix**.
- Hàm **suy luận** trên 1 video.

> **Gợi ý tài nguyên:** UCF50 tương đối nhẹ, nhưng tính Optical Flow on-the-fly vẫn tốn thời gian. Với máy yếu, giảm `flow_stack`, `num_segments`, hoặc bật **cache**.



**Giải thích (cell này):**  
- **Mục đích:** Giới thiệu tổng quan bài toán và những thành phần bạn sẽ có trong notebook.  
- **Đầu vào:** Không yêu cầu đầu vào.  
- **Đầu ra:** Không có.  
- **Kết quả kỳ vọng:** Bạn nắm được cấu trúc tổng thể notebook và các tùy chọn Fusion.


In [ ]:

# (Tùy chọn) Cài đặt thư viện nếu môi trường thiếu.
# Bạn có thể comment nếu đã có sẵn.
# !pip install opencv-python tqdm

import os
import math
import json
import time
import random
import shutil
from pathlib import Path
from dataclasses import dataclass, asdict

import cv2
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.models import resnet18

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report



**Giải thích:**  
- **Mục đích:** Import thư viện cần thiết; có cell cài đặt nếu thiếu.  
- **Đầu vào:** Không.  
- **Đầu ra:** Không.  
- **Kết quả kỳ vọng:** Môi trường sẵn sàng với `OpenCV`, `PyTorch`, `torchvision`, `tqdm`, `sklearn`.


In [ ]:

@dataclass
class Config:
    # Đường dẫn & IO
    DATA_ROOT: str = "/kaggle/input/ucf50/UCF50"   # Sửa theo nơi lưu dữ liệu (VD: local)
    OUTPUT_DIR: str = "./outputs_ucf50_fusion"
    LABELS_JSON: str = "labels_map.json"
    FLOW_CACHE_DIR: str = "./flow_cache"           # Cache *.npy cho optical flow (tùy chọn)

    # Dataloader
    IMG_SIZE: int = 224
    BATCH_SIZE: int = 16
    NUM_WORKERS: int = 2
    VAL_SPLIT: float = 0.2
    SEED: int = 1337

    # Lấy mẫu video
    NUM_SEGMENTS: int = 1          # số frame mẫu (cho RGB). Two-Stream 2D thường 1 frame/clip
    FLOW_STACK: int = 5            # số cặp (u,v) liên tiếp => 2*FLOW_STACK kênh
    FLOW_METHOD: str = "farneback" # "farneback" | "tvl1"
    FLOW_PRECOMPUTE: bool = True   # bật cache optical flow ra đĩa

    # Huấn luyện
    NUM_EPOCHS: int = 10
    LEARNING_RATE: float = 1e-3
    WEIGHT_DECAY: float = 1e-4
    SCHEDULER: str = "cosine"      # "cosine" | "step" | "plateau" | "none"
    STEP_SIZE: int = 5
    GAMMA: float = 0.1
    T_MAX: int = 10                # cho CosineAnnealingLR
    EARLY_STOP_PATIENCE: int = 5

    # Fusion
    FUSION: str = "late"           # "late" | "early_feature" | "early_channel"
    LATE_FUSION_WEIGHTS: tuple = (0.5, 0.5)  # (w_rgb, w_flow)
    # Nếu early_channel: in_channels = 3 + 2*FLOW_STACK
    # Nếu early_feature: concat features: [feat_rgb ; feat_flow]

    # Lưu & Log
    SAVE_BEST: bool = True
    BEST_MODEL_PATH: str = "best_model.pth"
    LOG_JSON: str = "train_log.json"

cfg = Config()

# Tạo thư mục xuất
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
os.makedirs(cfg.FLOW_CACHE_DIR, exist_ok=True)

# Cố định seed cho tái lập
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.SEED)
print(cfg)



**Giải thích:**  
- **Mục đích:** Khai báo toàn bộ cấu hình trong một `dataclass`.  
- **Đầu vào:** Không. Bạn sửa trực tiếp trong cell nếu cần (đặc biệt `DATA_ROOT`).  
- **Đầu ra:** Tạo thư mục `OUTPUT_DIR`, `FLOW_CACHE_DIR`. In ra cấu hình.  
- **Kết quả kỳ vọng:** Thay đổi cấu hình là trung tâm: kiểu fusion, scheduler, batch size, số epoch...


In [ ]:

def scan_ucf50(root: str):
    root = Path(root)
    classes = sorted([p.name for p in root.iterdir() if p.is_dir()])
    items = []
    for ci, cname in enumerate(classes):
        for v in (root/cname).glob("*.avi"):
            items.append({"path": str(v), "label_name": cname, "label": ci})
    return classes, items

classes, items = scan_ucf50(cfg.DATA_ROOT)
num_classes = len(classes)
print(f"Found {len(items)} videos across {num_classes} classes.")

# Lưu nhãn
labels_map = {i: c for i, c in enumerate(classes)}
with open(os.path.join(cfg.OUTPUT_DIR, cfg.LABELS_JSON), "w", encoding="utf-8") as f:
    json.dump(labels_map, f, ensure_ascii=False, indent=2)

# Stratified split theo lớp
from collections import defaultdict
by_class = defaultdict(list)
for it in items:
    by_class[it["label"]].append(it)

train_list, val_list = [], []
for k, vids in by_class.items():
    n = len(vids)
    idx = list(range(n))
    random.shuffle(idx)
    cut = int(n * (1.0 - cfg.VAL_SPLIT))
    for j in idx[:cut]:
        train_list.append(vids[j])
    for j in idx[cut:]:
        val_list.append(vids[j])

print(f"Train videos: {len(train_list)}, Val videos: {len(val_list)}")



**Giải thích:**  
- **Mục đích:** Dò tất cả video trong UCF50, tạo **nhãn** từ tên thư mục và **chia train/val** theo tỷ lệ trong `cfg.VAL_SPLIT`.  
- **Đầu vào:** `DATA_ROOT` trỏ tới cấu trúc `UCF50/ClassName/*.avi`.  
- **Đầu ra:** Danh sách `train_list`, `val_list` (mỗi phần tử gồm `path`, `label`, `label_name`), file `labels_map.json`.  
- **Kết quả kỳ vọng:** Thông tin số lượng video mỗi tập, số lớp.


In [ ]:

def read_video_frames(path, target_indices, resize_hw=None):
    '''Đọc một số frame theo chỉ số trong video. Trả về list ảnh BGR (np.uint8).'''
    cap = cv2.VideoCapture(path)
    frames = []
    if not cap.isOpened():
        cap.release()
        raise RuntimeError(f"Cannot open video: {path}")
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for ti in target_indices:
        idx = min(max(int(ti), 0), max(total-1, 0))
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if not ok:
            ok2, frame2 = cap.read()
            if not ok2:
                frame = np.zeros((resize_hw[0], resize_hw[1], 3), dtype=np.uint8) if resize_hw else None
            else:
                frame = frame2
        if resize_hw is not None and frame is not None:
            frame = cv2.resize(frame, (resize_hw[1], resize_hw[0]), interpolation=cv2.INTER_LINEAR)
        frames.append(frame)
    cap.release()
    return frames

def compute_flow_pair(prev_gray, next_gray, method="farneback"):
    if method == "farneback":
        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray,
                                            None, 0.5, 3, 15, 3, 5, 1.2, 0)
        return flow  # HxWx2 (u,v)
    elif method == "tvl1":
        tvl1 = cv2.optflow.DualTVL1OpticalFlow_create()
        flow = tvl1.calc(prev_gray, next_gray, None)
        return flow
    else:
        raise ValueError("FLOW_METHOD must be 'farneback' or 'tvl1'")

def flow_to_uv_stack(frames_gray, flow_stack=5, method="farneback"):
    '''Tính (u,v) cho flow_stack cặp khung hình liên tiếp quanh frame trung tâm.
       Trả về array shape (2*flow_stack, H, W), dtype float32.'''
    T = len(frames_gray)
    if T < flow_stack + 1:
        while len(frames_gray) < flow_stack + 1:
            frames_gray.append(frames_gray[-1])
        T = len(frames_gray)
    uv_list = []
    for i in range(flow_stack):
        f0 = frames_gray[i]
        f1 = frames_gray[i+1]
        flow = compute_flow_pair(f0, f1, method=method)  # HxWx2
        u, v = flow[..., 0], flow[..., 1]
        uv_list.append(u.astype(np.float32))
        uv_list.append(v.astype(np.float32))
    uv = np.stack(uv_list, axis=0)  # (2*flow_stack, H, W)
    uv = np.clip(uv, -20.0, 20.0) / 20.0  # [-1,1]
    return uv

def load_or_compute_flow_stack(video_path, center_idx, resize_hw, flow_stack, method, cache_dir):
    '''Tải từ cache hoặc tính flow quanh center_idx. Cache theo tên file.'''
    from pathlib import Path
    key = f"{Path(video_path).stem}_f{center_idx}_s{flow_stack}_{method}_{resize_hw[0]}x{resize_hw[1]}.npy"
    cache_path = Path(cache_dir)/key
    if cache_path.exists():
        return np.load(str(cache_path))
    frame_indices = [center_idx + i for i in range(flow_stack+1)]
    frames = read_video_frames(video_path, frame_indices, resize_hw)
    grays = [cv2.cvtColor(f, cv2.COLOR_BGR2GRAY) for f in frames]
    uv = flow_to_uv_stack(grays, flow_stack=flow_stack, method=method)
    np.save(str(cache_path), uv)
    return uv



**Giải thích:**  
- **Mục đích:** Định nghĩa utility đọc một số frame từ video và tính **Optical Flow** cho một **stack** cặp khung hình liên tiếp.  
- **Đầu vào:** `video_path`, chỉ số frame trung tâm, `FLOW_STACK`, kích thước `resize_hw`, `FLOW_METHOD`.  
- **Đầu ra:** Tensor `uv` dạng `(2*FLOW_STACK, H, W)` đã được cắt/chuẩn hóa về [-1, 1]. Có hỗ trợ **cache** dưới dạng `.npy`.  
- **Kết quả kỳ vọng:** Tối ưu tốc độ bằng cache; khi đổi `FLOW_METHOD` hoặc kích thước/stack thì cache khác nhau.


In [ ]:

class UCF50TwoStreamDataset(Dataset):
    def __init__(self, items, img_size=224, num_segments=1, flow_stack=5,
                 flow_method="farneback", flow_cache_dir=None, mode="train"):
        self.items = items
        self.H = img_size
        self.W = img_size
        self.num_segments = num_segments
        self.flow_stack = flow_stack
        self.flow_method = flow_method
        self.flow_cache_dir = flow_cache_dir
        self.mode = mode

        self.rgb_train_tf = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((self.H, self.W)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
        self.rgb_val_tf = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((self.H, self.W)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])

        self.to_tensor = transforms.ToTensor()

    def __len__(self):
        return len(self.items)

    def _sample_center(self, vpath):
        cap = cv2.VideoCapture(vpath)
        if not cap.isOpened():
            return 0, 1
        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        if total <= 1:
            return 0, 1
        ci = random.randint(1, max(total-2, 1))
        return ci, total

    def __getitem__(self, idx):
        rec = self.items[idx]
        vpath = rec["path"]
        label = rec["label"]

        center_idx, total = self._sample_center(vpath)

        rgb_frame = read_video_frames(vpath, [center_idx], resize_hw=(self.H, self.W))[0]  # BGR
        rgb = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2RGB)
        if self.mode == "train":
            rgb_t = self.rgb_train_tf(rgb)
        else:
            rgb_t = self.rgb_val_tf(rgb)

        if self.flow_cache_dir is not None:
            uv = load_or_compute_flow_stack(
                vpath, center_idx, (self.H, self.W),
                self.flow_stack, self.flow_method, self.flow_cache_dir
            )
        else:
            frames = read_video_frames(vpath, [center_idx + i for i in range(self.flow_stack+1)],
                                       resize_hw=(self.H, self.W))
            grays = [cv2.cvtColor(f, cv2.COLOR_BGR2GRAY) for f in frames]
            uv = flow_to_uv_stack(grays, self.flow_stack, self.flow_method)
        flow_t = torch.from_numpy(uv)  # (2*flow_stack, H, W)

        return rgb_t, flow_t, label



**Giải thích:**  
- **Mục đích:** Định nghĩa `Dataset` trả về bộ **(RGB tensor, FLOW tensor, label)** cho mỗi mẫu video.  
- **Đầu vào:** `items` (danh sách video), `img_size`, `flow_stack`, `flow_method`, `flow_cache_dir`.  
- **Đầu ra:** `rgb_t` dạng `(3,H,W)`, `flow_t` dạng `(2*FLOW_STACK,H,W)`, `label` (int).  
- **Kết quả kỳ vọng:** Dataloader phục vụ 2 stream (RGB & Flow) cho các cơ chế Fusion.


In [ ]:

train_ds = UCF50TwoStreamDataset(
    train_list, img_size=cfg.IMG_SIZE, num_segments=cfg.NUM_SEGMENTS,
    flow_stack=cfg.FLOW_STACK, flow_method=cfg.FLOW_METHOD,
    flow_cache_dir=(cfg.FLOW_CACHE_DIR if cfg.FLOW_PRECOMPUTE else None),
    mode="train"
)
val_ds = UCF50TwoStreamDataset(
    val_list, img_size=cfg.IMG_SIZE, num_segments=cfg.NUM_SEGMENTS,
    flow_stack=cfg.FLOW_STACK, flow_method=cfg.FLOW_METHOD,
    flow_cache_dir=(cfg.FLOW_CACHE_DIR if cfg.FLOW_PRECOMPUTE else None),
    mode="val"
)

train_loader = DataLoader(train_ds, batch_size=cfg.BATCH_SIZE, shuffle=True,
                          num_workers=cfg.NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=cfg.BATCH_SIZE, shuffle=False,
                        num_workers=cfg.NUM_WORKERS, pin_memory=True)

len(train_ds), len(val_ds)



**Giải thích:**  
- **Mục đích:** Khởi tạo `Dataset` và `DataLoader` cho train/val.  
- **Đầu vào:** `train_list`, `val_list`; cấu hình batch size, workers.  
- **Đầu ra:** `train_loader`, `val_loader`.  
- **Kết quả kỳ vọng:** Có thể lặp qua batch gồm `(rgb, flow, label)`.


In [ ]:

def inflate_conv1_weight(conv1: nn.Conv2d, new_in_channels: int):
    '''Mở rộng conv1 để nhận nhiều kênh > 3 bằng cách lặp/avg trọng số ban đầu.'''
    old_w = conv1.weight.data  # (out_c, in_c, k, k)
    out_c, in_c, kh, kw = old_w.shape
    if new_in_channels == in_c:
        return conv1
    new_w = torch.zeros((out_c, new_in_channels, kh, kw))
    for oc in range(out_c):
        for ic in range(new_in_channels):
            new_w[oc, ic] = old_w[oc, ic % in_c]
    conv1.in_channels = new_in_channels
    conv1.weight = nn.Parameter(new_w)
    return conv1

class EarlyChannelResNet(nn.Module):
    '''Early-Channel Fusion: ghép RGB + Flow theo kênh => một backbone duy nhất.'''
    def __init__(self, num_classes, in_channels):
        super().__init__()
        self.backbone = resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.conv1 = inflate_conv1_weight(self.backbone.conv1, in_channels)
        in_feat = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(in_feat, num_classes)

    def forward(self, x):  # x: (B, C, H, W)
        return self.backbone(x)

class TwoBackboneEarlyFeature(nn.Module):
    '''Early-Feature Fusion: 2 backbone riêng -> concat features -> classifier.'''
    def __init__(self, num_classes, flow_in_channels):
        super().__init__()
        self.rgb_net = resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        self.flow_net = resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        self.flow_net.conv1 = inflate_conv1_weight(self.flow_net.conv1, flow_in_channels)
        self.rgb_net.fc = nn.Identity()
        self.flow_net.fc = nn.Identity()
        in_feat = 512 + 512
        self.fc = nn.Linear(in_feat, num_classes)

    def forward(self, rgb, flow):
        fr = self.rgb_net(rgb)   # (B,512)
        ff = self.flow_net(flow) # (B,512)
        f = torch.cat([fr, ff], dim=1)
        logits = self.fc(f)
        return logits

class LateFusionModel(nn.Module):
    '''Late Fusion: 2 backbone riêng -> logits riêng -> trộn theo trọng số.'''
    def __init__(self, num_classes, flow_in_channels, w_rgb=0.5, w_flow=0.5):
        super().__init__()
        self.rgb_net = resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        self.flow_net = resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        self.flow_net.conv1 = inflate_conv1_weight(self.flow_net.conv1, flow_in_channels)
        self.rgb_head = nn.Linear(self.rgb_net.fc.in_features, num_classes)
        self.flow_head = nn.Linear(self.flow_net.fc.in_features, num_classes)
        self.w_rgb = w_rgb
        self.w_flow = w_flow
        self.rgb_net.fc = nn.Identity()
        self.flow_net.fc = nn.Identity()

    def forward(self, rgb, flow):
        fr = self.rgb_net(rgb)    # (B,512)
        ff = self.flow_net(flow)  # (B,512)
        lr = self.rgb_head(fr)    # (B,C)
        lf = self.flow_head(ff)   # (B,C)
        logits = self.w_rgb * lr + self.w_flow * lf
        return logits, lr, lf



**Giải thích:**  
- **Mục đích:** Khai báo 3 biến thể mô hình Fusion:
  - `EarlyChannelResNet`: ghép **RGB + Flow theo kênh** → một backbone.
  - `TwoBackboneEarlyFeature`: 2 backbone, **nối đặc trưng** trước classifier.
  - `LateFusionModel`: 2 backbone, **trộn logits** theo trọng số `(w_rgb, w_flow)`.
- **Đầu vào:** Tensor RGB `(B,3,H,W)`, Tensor Flow `(B,2*FLOW_STACK,H,W)` (tùy biến thể).  
- **Đầu ra:** `logits` phân lớp (và `lr, lf` cho late fusion).  
- **Kết quả kỳ vọng:** Linh hoạt chọn chiến lược Fusion bằng `cfg.FUSION`.


In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

flow_in_ch = 2 * cfg.FLOW_STACK

if cfg.FUSION == "early_channel":
    model = EarlyChannelResNet(num_classes=num_classes, in_channels=3 + flow_in_ch).to(device)
elif cfg.FUSION == "early_feature":
    model = TwoBackboneEarlyFeature(num_classes=num_classes, flow_in_channels=flow_in_ch).to(device)
elif cfg.FUSION == "late":
    w_rgb, w_flow = cfg.LATE_FUSION_WEIGHTS
    model = LateFusionModel(num_classes=num_classes, flow_in_channels=flow_in_ch,
                            w_rgb=w_rgb, w_flow=w_flow).to(device)
else:
    raise ValueError("cfg.FUSION must be one of: early_channel | early_feature | late")

optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.LEARNING_RATE, weight_decay=cfg.WEIGHT_DECAY)
if cfg.SCHEDULER == "cosine":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.T_MAX)
elif cfg.SCHEDULER == "step":
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=cfg.STEP_SIZE, gamma=cfg.GAMMA)
elif cfg.SCHEDULER == "plateau":
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)
else:
    scheduler = None

criterion = nn.CrossEntropyLoss()

class EarlyStopper:
    def __init__(self, patience=5):
        self.patience = patience
        self.best = -1.0
        self.count = 0
    def step(self, metric):
        if metric > self.best:
            self.best = metric
            self.count = 0
            return True
        else:
            self.count += 1
            return False
    def should_stop(self):
        return self.count >= self.patience

early_stopper = EarlyStopper(cfg.EARLY_STOP_PATIENCE)



**Giải thích:**  
- **Mục đích:** Khởi tạo mô hình theo `cfg.FUSION`, cùng **optimizer**, **scheduler**, và **early stopping**.  
- **Đầu vào:** `cfg` (hệ số, kiểu fusion, learning rate, v.v.).  
- **Đầu ra:** `model`, `optimizer`, `scheduler`, `criterion`, `early_stopper`.  
- **Kết quả kỳ vọng:** Sẵn sàng huấn luyện theo chiến lược đã chọn.


In [ ]:

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total, correct, loss_sum = 0, 0, 0.0
    for rgb, flow, y in tqdm(loader, desc="Train", leave=False):
        rgb = rgb.to(device)
        flow = flow.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        if isinstance(model, EarlyChannelResNet):
            x = torch.cat([rgb, flow], dim=1)  # (B, 3+2*flow_stack, H, W)
            logits = model(x)
        elif isinstance(model, TwoBackboneEarlyFeature):
            logits = model(rgb, flow)
        elif isinstance(model, LateFusionModel):
            logits, _, _ = model(rgb, flow)
        else:
            raise RuntimeError("Unknown model type")

        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            pred = logits.argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
            loss_sum += loss.item() * y.size(0)

    return loss_sum/total, correct/total

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    all_y, all_pred = [], []
    for rgb, flow, y in tqdm(loader, desc="Val", leave=False):
        rgb = rgb.to(device)
        flow = flow.to(device)
        y = y.to(device)
        if isinstance(model, EarlyChannelResNet):
            x = torch.cat([rgb, flow], dim=1)
            logits = model(x)
        elif isinstance(model, TwoBackboneEarlyFeature):
            logits = model(rgb, flow)
        elif isinstance(model, LateFusionModel):
            logits, _, _ = model(rgb, flow)
        else:
            raise RuntimeError("Unknown model type")

        loss = criterion(logits, y)
        pred = logits.argmax(dim=1)

        correct += (pred == y).sum().item()
        total += y.size(0)
        loss_sum += loss.item() * y.size(0)

        all_y.extend(y.cpu().numpy().tolist())
        all_pred.extend(pred.cpu().numpy().tolist())

    acc = correct/total if total > 0 else 0.0
    return loss_sum/max(total,1), acc, np.array(all_y), np.array(all_pred)



**Giải thích:**  
- **Mục đích:** Vòng lặp **train** và **evaluate** chia sẻ logic cho ba biến thể model.  
- **Đầu vào:** `model`, `DataLoader`, `optimizer`, `criterion`.  
- **Đầu ra:** Loss trung bình, Accuracy, (ở `evaluate` có thêm `y_true`, `y_pred`).  
- **Kết quả kỳ vọng:** Theo dõi tiến trình huấn luyện và tổng hợp chỉ số.


In [ ]:

best_acc = -1.0
log_hist = []

for epoch in range(1, cfg.NUM_EPOCHS+1):
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, y_true, y_pred = evaluate(model, val_loader, criterion, device)

    if cfg.SCHEDULER == "plateau":
        scheduler.step(val_acc)
    elif cfg.SCHEDULER in ["cosine", "step"] and scheduler is not None:
        scheduler.step()

    rec = {
        "epoch": epoch,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc,
        "lr": optimizer.param_groups[0]["lr"],
        "time_sec": round(time.time()-t0, 2)
    }
    log_hist.append(rec)
    print(f"[Epoch {epoch:02d}] "
          f"train_loss={train_loss:.4f} acc={train_acc:.4f} | "
          f"val_loss={val_loss:.4f} acc={val_acc:.4f} | lr={rec['lr']:.2e} | "
          f"time={rec['time_sec']}s")

    improved = val_acc > best_acc
    if improved and cfg.SAVE_BEST:
        best_acc = val_acc
        torch.save(model.state_dict(), os.path.join(cfg.OUTPUT_DIR, cfg.BEST_MODEL_PATH))
        with open(os.path.join(cfg.OUTPUT_DIR, "best_epoch.txt"), "w") as f:
            f.write(f"best_epoch={epoch}\nval_acc={val_acc:.4f}\n")

    if early_stopper.step(val_acc) is False and early_stopper.should_stop():
        print("Early stopping triggered.")
        break

with open(os.path.join(cfg.OUTPUT_DIR, cfg.LOG_JSON), "w") as f:
    json.dump(log_hist, f, indent=2)

print("Training done. Best val acc:", best_acc)



**Giải thích:**  
- **Mục đích:** Tiến hành huấn luyện đầy đủ, theo dõi loss/accuracy, cập nhật LR, lưu **best model**, và **early stopping**.  
- **Đầu vào:** Cấu hình và các thành phần đã khởi tạo trước đó.  
- **Đầu ra:** File `best_model.pth`, `best_epoch.txt`, và log `train_log.json` trong `OUTPUT_DIR`.  
- **Kết quả kỳ vọng:** Huấn luyện dừng sớm nếu không tiến bộ; bạn có mô hình tốt nhất trên tập validation.


In [ ]:

@torch.no_grad()
def evaluate_and_report(model, loader, device, class_names):
    model.eval()
    all_y, all_pred = [], []
    for rgb, flow, y in tqdm(loader, desc="Eval", leave=False):
        rgb = rgb.to(device)
        flow = flow.to(device)
        y = y.to(device)
        if isinstance(model, EarlyChannelResNet):
            x = torch.cat([rgb, flow], dim=1)
            logits = model(x)
        elif isinstance(model, TwoBackboneEarlyFeature):
            logits = model(rgb, flow)
        elif isinstance(model, LateFusionModel):
            logits, _, _ = model(rgb, flow)
        pred = logits.argmax(dim=1)
        all_y.extend(y.cpu().numpy().tolist())
        all_pred.extend(pred.cpu().numpy().tolist())
    all_y = np.array(all_y)
    all_pred = np.array(all_pred)
    print(classification_report(all_y, all_pred, target_names=class_names, digits=3))
    cm = confusion_matrix(all_y, all_pred, labels=list(range(len(class_names))))
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest')
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.colorbar()
    plt.tight_layout()
    plt.show()
    return cm

best_path = os.path.join(cfg.OUTPUT_DIR, cfg.BEST_MODEL_PATH)
if os.path.exists(best_path):
    state = torch.load(best_path, map_location=device)
    model.load_state_dict(state, strict=False)
    _ = evaluate_and_report(model, val_loader, device, classes)
else:
    print("Best model not found; please train first.")



**Giải thích:**  
- **Mục đích:** Sinh **classification report** (precision/recall/F1) và **Confusion Matrix**.  
- **Đầu vào:** `model` đã huấn luyện, `val_loader`, danh sách `classes`.  
- **Đầu ra:** In báo cáo và vẽ ma trận nhầm lẫn.  
- **Kết quả kỳ vọng:** Đánh giá trực quan, nhận biết lớp nào hay nhầm.


In [ ]:

@torch.no_grad()
def predict_single_video(model, video_path, cfg):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.eval()
    cap = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.isOpened() else 2
    cap.release()
    ci = min(max(total//2, 1), max(total-2, 1))

    rgb_frame = read_video_frames(video_path, [ci], resize_hw=(cfg.IMG_SIZE, cfg.IMG_SIZE))[0]
    rgb = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2RGB)
    tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((cfg.IMG_SIZE, cfg.IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
    rgb_t = tf(rgb).unsqueeze(0).to(device)

    uv = load_or_compute_flow_stack(
        video_path, ci, (cfg.IMG_SIZE, cfg.IMG_SIZE), cfg.FLOW_STACK, cfg.FLOW_METHOD, cfg.FLOW_CACHE_DIR
    )
    flow_t = torch.from_numpy(uv).unsqueeze(0).to(device)  # (1,2*stack,H,W)

    if isinstance(model, EarlyChannelResNet):
        x = torch.cat([rgb_t, flow_t], dim=1)
        logits = model(x)
    elif isinstance(model, TwoBackboneEarlyFeature):
        logits = model(rgb_t, flow_t)
    elif isinstance(model, LateFusionModel):
        logits, _, _ = model(rgb_t, flow_t)

    prob = F.softmax(logits, dim=1)[0].cpu().numpy()
    pred_idx = int(np.argmax(prob))
    return pred_idx, prob

# Ví dụ sử dụng (sau khi train):
# pred_idx, prob = predict_single_video(model, "/kaggle/input/ucf50/UCF50/BenchPress/v_BenchPress_g08_c02.avi", cfg)
# print("Dự đoán:", classes[pred_idx])



**Giải thích:**  
- **Mục đích:** Suy luận trên **một video** duy nhất, trả về lớp dự đoán và phân bố xác suất.  
- **Đầu vào:** `video_path` tới tệp `.avi`, `cfg`.  
- **Đầu ra:** `pred_idx` (int), `prob` (mảng xác suất).  
- **Kết quả kỳ vọng:** Dễ dàng kiểm thử nhanh một video bất kỳ.



## Notes & Tips

- **Cấu trúc dữ liệu UCF50** mong đợi:
  ```
  UCF50/
    ClassA/
      video1.avi
      video2.avi
      ...
    ClassB/
      v_*.avi
      ...
    ...
  ```
- **Tốc độ:** Tính Optical Flow on-the-fly là nặng. Hãy cân nhắc:
  - Giảm `FLOW_STACK` (ví dụ, 3)
  - Giảm `IMG_SIZE` (ví dụ, 168 hoặc 128)
  - Chuyển `FLOW_METHOD` sang `"farneback"` (nhanh hơn TV-L1)
  - **Bật cache** (`FLOW_PRECOMPUTE=True`), notebook sẽ tự lưu/tải `.npy`.
- **Ký hiệu Fusion:**
  - *Early-Channel*: đơn giản, nhanh inference; nhưng phụ thuộc sửa conv1.
  - *Early-Feature*: linh hoạt ở mức đặc trưng; classifier chung.
  - *Late*: đơn giản, có thể huấn luyện 2 nhánh độc lập; dễ điều chỉnh trọng số.
- **Scheduler & EarlyStopping:**
  - `cosine` thường mượt; `step` giúp giảm LR theo mốc; `plateau` bám theo `val_acc`.
  - `EARLY_STOP_PATIENCE` nên chọn 3–8 tùy dữ liệu.
- **Đánh giá:** UCF50 hay dùng cross-subject splits; ở đây ta **stratified random split** để đơn giản.
- **Tái lập:** cố định `SEED`; chú ý `cudnn.benchmark=False` để nhất quán.



**Giải thích:**  
- **Mục đích:** Ghi chú vận hành, tối ưu và giải thích các biến thể fusion.  
- **Đầu vào:** Không.  
- **Đầu ra:** Không.  
- **Kết quả kỳ vọng:** Bạn có đủ gợi ý để tùy chỉnh cho máy và mục tiêu của mình.
